In [3]:
# Imports and environment detection
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import datetime as dt
import talib

In [4]:
TICKERS = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA']

## Load Dataset

In [ ]:
def read_stock_csv(ticker, data_dir=Path('../data')):
    path = data_dir / f'{ticker}.csv'
    if not path.exists():
        raise FileNotFoundError(f'File not found: {path}')
    df = pd.read_csv(path, parse_dates=['Date'], infer_datetime_format=True)
    df = df.sort_values('Date').reset_index(drop=True)
    return df

print(read_stock_csv('AAPL').head()) # testing load

        Date     Close      High       Low      Open      Volume
0 2009-01-02  2.721686  2.730385  2.554037  2.575630   746015200
1 2009-01-05  2.836553  2.884539  2.780469  2.794266  1181608400
2 2009-01-06  2.789767  2.914229  2.770872  2.877641  1289310400
3 2009-01-07  2.729484  2.774170  2.706990  2.753477   753048800
4 2009-01-08  2.780169  2.793666  2.700393  2.712090   673500800


C:\Users\Abel Tesfa\AppData\Local\Temp\ipykernel_34596\3089814550.py:5: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(path, parse_dates=['Date'], infer_datetime_format=True)


# Apply Analysis Indicators with TALIB

In [7]:
# Indicator functions (RSI, MACD, Bollinger, ATR, OBV, ADX, Stochastic)
def compute_rsi(close, period=14):
    return talib.RSI(close.values, timeperiod=period)

def compute_macd(close, fast=12, slow=26, signal=9):
    return talib.MACD(close.values, fastperiod=fast, slowperiod=slow, signalperiod=signal)

def compute_bollinger(close, length=20, stds=2):
    ma = close.rolling(window=length).mean()
    std = close.rolling(window=length).std()
    upper = ma + stds * std
    lower = ma - stds * std
    return ma, upper, lower

def compute_atr(df, length=14):
    high = df['High']
    low = df['Low']
    close = df['Close']
    tr1 = high - low
    tr2 = (high - close.shift()).abs()
    tr3 = (low - close.shift()).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.rolling(window=length).mean()
    return atr

def compute_obv(df):
    obv = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
    return obv

def compute_stochastic(df, k=14, d=3):
    low_min = df['Low'].rolling(k).min()
    high_max = df['High'].rolling(k).max()
    stoch_k = 100 * (df['Close'] - low_min) / (high_max - low_min)
    stoch_d = stoch_k.rolling(d).mean()
    return stoch_k, stoch_d

def compute_adx(df, n=14):
    return talib.ADX(df['High'].values, df['Low'].values, df['Close'].values, timeperiod=n)

In [8]:
def add_indicators(df):
    df = df.copy()
    close = df['Close']
    df['SMA_20'] = close.rolling(window=20).mean()
    df['EMA_20'] = close.ewm(span=20, adjust=False).mean()
    df['RSI_14'] = compute_rsi(close, 14)
    macd, macd_signal, macd_hist = compute_macd(close)
    df['MACD'] = macd
    df['MACD_signal'] = macd_signal
    df['MACD_hist'] = macd_hist
    ma, upper, lower = compute_bollinger(close)
    df['BB_MA'] = ma
    df['BB_upper'] = upper
    df['BB_lower'] = lower
    df['ATR_14'] = compute_atr(df, 14)
    df['OBV'] = compute_obv(df)
    stoch_k, stoch_d = compute_stochastic(df)
    df['STOCH_K'] = stoch_k
    df['STOCH_D'] = stoch_d
    df['ADX_14'] = compute_adx(df, 14)
    return df